## Getting top bollywood movies form IMDB using webscraping

In [1]:
# Use beautifulSoup to get the latest data

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [3]:
# UDF to generate urls

def url_gen(a):
#     a simple miner assist tool to generate URLs 
#     iterater from range(1, n) where n == R
    return 'https://www.imdb.com/search/title/?countries=in&locations=India&count=250&start=' \
                + str( (251) + (a * 250) ) \
                + '&ref_=adv_nxt'

In [4]:
# define empty lists to fetch scraped data from the miner
bollywood_title_all = []
bollywood_year_all = []
bollywood_title_links_all = []

In [5]:
def miner(url = 'https://www.imdb.com/search/title/?countries=in&locations=India&count=250'
          , bollywood_title_all = bollywood_title_all
          , bollywood_year_all = bollywood_year_all
          , bollywood_title_links_all = bollywood_title_links_all):

    # The miner takes in a default URL which is the 1st page to be mined.
    # for subsequent mines, we will use the url_gen function to pass in new urls.
    # the miner takes in the empty lists by default and updates them through each iteration.
    
    # Downloading imdb top 250 movie's data
    url = url
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')

    # use select method and enter the selector path of the element
    bollywood_title = (soup.select('div.lister-item-content > h3 > a'))
    bollywood_year = (soup.select('div.lister-item-content > h3 > span.lister-item-year.text-muted.unbold'))
    bollywood_title_links = [a.attrs.get('href') for a in soup.select('div.lister-item-content > h3 > a')]

    # convert bs4.element into string
    for i in range(0, len(bollywood_title)):
        bollywood_title[i] = bollywood_title[i].text

    for i in range(0, len(bollywood_year)):
        bollywood_year[i] = bollywood_year[i].text

    # Update the empty lists with new mined data in this iteration
    bollywood_title_all += bollywood_title
    bollywood_year_all += bollywood_year
    bollywood_title_links_all += bollywood_title_links 

In [ ]:
try:
    # call the miner with default page
    miner()
    print('mined no.', 1, 'page' )

    # iterate over the next i pages pages:
    for i in range(1, 50):
        miner(url = url_gen(i))
        print('mined no.', i + 1, 'page' )

except: 
    print("MINING ERROR !! DATA NOT MINED")

mined no. 1 page
mined no. 2 page
mined no. 3 page
mined no. 4 page
mined no. 5 page
mined no. 6 page
mined no. 7 page
mined no. 8 page
mined no. 9 page
mined no. 10 page
mined no. 11 page
mined no. 12 page
mined no. 13 page


In [ ]:
dict_for_df = {"title":bollywood_title_all, "release_year":bollywood_year_all, "titleId":bollywood_title_links_all}

In [ ]:
scraped_bollywood = pd.DataFrame(dict_for_df)
scraped_bollywood.release_year = scraped_bollywood.release_year.str.extract('(\d+)')
scraped_bollywood.titleId = scraped_bollywood.titleId.str.extract('(\d+)')

In [ ]:
scraped_bollywood

In [ ]:
scraped_bollywood.describe().T

In [ ]:
title_basics_raw = pd.read_csv("compressed_tsv/title.basics.tsv.gz" , sep='\t', low_memory = False)

In [ ]:
title_basics_raw.tconst = title_basics_raw.tconst.str.extract('(\d+)')
title_basics_raw

In [ ]:
if title_basics_raw[title_basics_raw.tconst == '9614490'].shape >(1,1):
    print("Successfully mined and reconciled")
else:
    print("FAILED to reconcile")

In [ ]:
!ls

In [ ]:
# save the df as a csv on local machine
scraped_bollywood.to_csv('mined_csv/scraped_bollywood.csv')

In [ ]:
print("saved csv to local drive")